In [1]:
%env CUDA_VISIBLE_DEVICES 0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
from tqdm.notebook import tqdm
from utils import PiiNERPipeline
from datasets import load_dataset

pipeline = PiiNERPipeline('bigcode/bigcode-encoder-pii-ner',
                          batch_size=32,
                          window_size=512,
                          device=0,
                          num_workers=2,
                          use_auth_token=True)
dataset = load_dataset('bigcode/pii-for-code-v2', use_auth_token=True)['train']

/home/monty/projects/pii-ner/utils/misc.py:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  _seqeval_metric = load_metric("seqeval")
Using the latest cached version of the module from /home/monty/.cache/huggingface/modules/datasets_modules/metrics/seqeval/c8563af43bdce095d0f9e8b8b79c9c96d5ea5499b3bf66f90301c9cb82910f11 (last modified on Thu Feb 16 17:58:29 2023) since it couldn't be found locally at seqeval, or remotely on the Hugging Face Hub.
Found cached dataset parquet (/home/monty/.cache/huggingface/datasets/bigcode___parquet/bigcode--pii-for-code-v2-e4dbf8dee82c409a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
iterator = pipeline(dataset)
processed_dataset = list(tqdm(iterator, total=len(dataset), desc='Detect PII entities'))

Detect PII entities:   0%|          | 0/400 [00:00<?, ?it/s]

Too many dataloader workers: 2 (max is dataset.n_shards=1). Stopping 1 dataloader workers.
To parallelize data loading, we give each process some shards (or data sources) to process. Therefore it's unnecessary to have a number of workers greater than dataset.n_shards=1. To enable more parallelism, please split the dataset in more files than 1.


In [5]:
processed_dataset[0]['entities']

[{'tag': 'NAME',
  'start': 23,
  'end': 36,
  'value': 'Davis E. King',
  'context': '// Copyright (C) 2003  Davis E. King (davis@dlib.net), Miguel Grinberg\n// License: Boo',
  'score': 0.9986383},
 {'tag': 'EMAIL',
  'start': 38,
  'end': 52,
  'value': 'davis@dlib.net',
  'context': '// Copyright (C) 2003  Davis E. King (davis@dlib.net), Miguel Grinberg\n// License: Boost Software Lice',
  'score': 0.9993752},
 {'tag': 'NAME',
  'start': 55,
  'end': 70,
  'value': 'Miguel Grinberg',
  'context': 'opyright (C) 2003  Davis E. King (davis@dlib.net), Miguel Grinberg\n// License: Boost Software License   See LICENSE.',
  'score': 0.99788713}]

## Do some profiling here

In [6]:
!pip install line_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 1.3 MB/s eta 0:00:00a 0:00:01


In [3]:
%load_ext line_profiler

In [4]:
%lprun -f pipeline._predict_iterator list(tqdm( pipeline(dataset), total=len(dataset), desc='Detect PII entities'))

Detect PII entities:   0%|          | 0/400 [00:00<?, ?it/s]

Too many dataloader workers: 2 (max is dataset.n_shards=1). Stopping 1 dataloader workers.
To parallelize data loading, we give each process some shards (or data sources) to process. Therefore it's unnecessary to have a number of workers greater than dataset.n_shards=1. To enable more parallelism, please split the dataset in more files than 1.


    Timer unit: 1e-09 s

    Total time: 289.552 s
    File: pii-ner/utils/pipeline.py
    Function: _predict_iterator at line 159

    Line #      Hits         Time  Per Hit   % Time  Line Contents
    ==============================================================
       159                                               def _predict_iterator(self, inputs: Dataset, batch_size: int):
       160         1     382949.0 382949.0      0.0          loader = self._get_pipeline_dataloader(inputs, self.tokenizer,
       161         1        214.0    214.0      0.0                                                 batch_size=batch_size,
       162         1        427.0    427.0      0.0                                                 window_size=self.window_size,
       163         1        242.0    242.0      0.0                                                 window_overlap=self.window_overlap,
       164         1        247.0    247.0      0.0                                                 num_workers=self.num_workers)
       165                                           
       166         1 1902225579.0 1902225579.0      0.7          self.model.to(self.device)
       167                                           
       168         1       2833.0   2833.0      0.0          processing_iterator = self.process_inputs(loader)
       169       400 283532077631.0 708830194.1     97.9          for processed in self.combine_chunked_inputs(processing_iterator):
       170       400 4116858091.0 10292145.2      1.4              yield self.extract_entities(text=processed['text'],
       171       400     157521.0    393.8      0.0                                          logits=processed['logits'],
       172       400     162483.0    406.2      0.0                                          offset_mapping=processed['offset_mapping'])
       173                                                   self.model.to('cpu')

In [15]:
from utils.pipeline import PipelineIterator
pipeline.num_workers = 0
%lprun -f pipeline.forward list(tqdm( pipeline(dataset.from_list(list(dataset)[:20])), total=20, desc='Detect PII entities'))

Detect PII entities:   0%|          | 0/20 [00:00<?, ?it/s]

    Timer unit: 1e-09 s

    Total time: 12.1679 s
    File: pii-ner/utils/pipeline.py
    Function: forward at line 100

    Line #      Hits         Time  Per Hit   % Time  Line Contents
    ==============================================================
       100                                               def forward(self, model_inputs, **forward_params):
       101         7     376197.0  53742.4      0.0          with self.device_placement():
       102         7     585588.0  83655.4      0.0              inference_context = self.get_inference_context()
       103         7     142410.0  20344.3      0.0              with inference_context():
       104         7 5039463571.0 719923367.3     41.4                  model_inputs = self._ensure_tensor_on_device(model_inputs, device=self.device)
       105         7   86368434.0 12338347.7      0.7                  model_outputs = self._forward(model_inputs, **forward_params)
       106         7 7040784741.0 1005826391.6     57.9                  model_outputs = self._ensure_tensor_on_device(model_outputs, device=torch.device("cpu"))
       107         7     211824.0  30260.6      0.0                  model_outputs = {name: tensor.numpy() if isinstance(tensor, torch.Tensor) else tensor
       108         7       6737.0    962.4      0.0                                   for name, tensor in model_outputs.items()}
       109                                           
       110         7       1324.0    189.1      0.0          return model_outputs

### 5 seconds out 12 (41% of time) consumed on transferring from cpu to cuda 
### 7 seconds out 12 (58% of time) consumed on transferring from cuda to cpu 
### so, most of the time consumption is just a bus bottleneck =/